In [1]:
import pysam
from Bio import SeqIO
print("usage: python filter_bam.py <sample.bam> <genome.fa> <motif>")
print("example: python filter_bam.py markDup/L7_S2.bam hg38.primary.fa GATC")

usage: python filter_bam.py <sample.bam> <genome.fa> <motif>
example: python filter_bam.py markDup/L7_S2.bam hg38.primary.fa GATC


In [2]:
# read genome fasta
genome_dict = {rec.id : rec.seq for rec in SeqIO.parse("hg38.primary.fa", "fasta")}
#chrom = str(seq_dict['chr1'])

In [5]:
rawbam = pysam.AlignmentFile("markDup/test.L7.bam", "rb")
fltbam =  pysam.AlignmentFile("filtered_bam/test.L7.bam", "wb", template=rawbam)

def contain_motif(motif="GATC", seq="AcTggatc"):
    motif = motif.upper()
    seq = seq.upper()
    if motif in seq:
        return True
    else:
        return False    

    
# go through rawbam, filter, then output fltbam
iter = rawbam.fetch()
n1 = 0
n2 = 0
for x in iter:
    if x.is_secondary:
        continue
    if x.is_unmapped:
        continue
    if x.mapping_quality < 20:
        continue
    n1 += 1
    
    chrom = genome_dict[x.reference_name]
    if x.is_reverse:
        pos = x.reference_end
        seq = chrom[pos-20:pos+21] # RC same as Seq for DpnII
    else:
        pos = x.reference_start
        seq = chrom[pos-21:pos+20]
    
    if contain_motif("GATC", seq):
        n2 += 1
        fltbam.write(x)  

print("n1 = ", n1, ", n2 = ", n2, "n2/n1", n2/n1)
rawbam.close()
fltbam.close()

n1 =  57003 , n2 =  16220 n2/n1 0.28454642738101504
